## Dit is de API voor project 4 VisuStore


In [ ]:
{
    "Inlog DB VM"{
        "IP"   : "192.168.133.123",
        "User" : "student",
        "Pass" : "VisuPass!@"
    }

    "DB Admin user"{
        "User" : "vsadmin",
        "Pass" : "visupapa"
    }
}

Imports:

In [18]:
import os
from dotenv import load_dotenv
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

Dit volgende stukje maakt de flask app en zet de connectie string en inlog. In de config wordt de inlog naam en wachtwoord uit het bestand .env gevraagd


In [2]:
load_dotenv(override=True)

print(os.getenv('DB_USER'))
print(os.getenv('DB_PASS'))
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = (
    f"mysql+pymysql://"
    f"{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

vsadmin
visupapa


In [ ]:
from sqlalchemy import create_engine, MetaData, text

engine = create_engine(
    f"mysql+pymysql://"
    f"{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
)
print(os.getenv('DB_USER'))

metadata = MetaData()
# Leest alle tabellen in de database in
metadata.reflect(bind=engine)

def insert_into(table_name: str, data: dict):
    """
    Insert een rij in de opgegeven tabel via SQLAlchemy Core.
    """
    # 1. Table-object ophalen
    if table_name not in metadata.tables:
        raise ValueError(f"Tabel '{table_name}' niet gevonden in metadata.")
    table = metadata.tables[table_name]

    # 2. Insert-statement bouwen
    stmt = table.insert().values(**data)

    # 3. Uitvoeren
    with engine.connect() as conn:
        conn.execute(stmt)
        conn.commit()
        print(f"1 rij toegevoegd aan {table_name}.")

# Voorbeeldgebruik:
# insert_into("warehouses", {
#     "name": "East Hub",
#     "address": "Laan 10",
#     "city": "Utrecht",
#     "postal_code": "3500BB"
# })
# insert_into("zones", {
#     "zone_id": 1,
#     "warehouse_id": 1,
#     "zone_name": "Zone A",
#     "zone_type": "Storage"
# })

vsadmin
1 rij toegevoegd aan warehouses.
1 rij toegevoegd aan zones.


In [ ]:
from sqlalchemy import create_engine, text

engine = create_engine(
    f"mysql+pymysql://"
    f"{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
)
print(os.getenv('DB_USER'))

def delete_from(table_name: str, where_clauses: dict):
    # 1. WHERE-expressie samenstellen
    conditions = " AND ".join(f"{col} = :{col}" for col in where_clauses.keys())

    # 2. Query bouwen
    query = text(f"DELETE FROM {table_name} WHERE {conditions}")

    # 3. Uitvoeren
    with engine.connect() as conn:
        result = conn.execute(query, where_clauses)
        conn.commit()
        print(f"{result.rowcount} rij(en) verwijderd uit {table_name}.")

# Voorbeeld:
# delete_from(
#     table_name="warehouses",
#     where_clauses={"warehouse_id": 5}
# )


vsadmin
1 rij(en) verwijderd uit warehouses.


In [ ]:
from sqlalchemy import create_engine, text

engine = create_engine(
    f"mysql+pymysql://"
    f"{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
)
print(os.getenv('DB_USER'))

def update_table(table_name: str, update_values: dict, where_clauses: dict):

    # 1. SET-expressie samenstellen
    set_expr = ", ".join(f"{col} = :upd_{col}" for col in update_values.keys())
    # 2. WHERE-expressie samenstellen
    where_expr = " AND ".join(f"{col} = :where_{col}" for col in where_clauses.keys())
    
    # 3. Query bouwen
    query = text(f"""
        UPDATE {table_name}
        SET {set_expr}
        WHERE {where_expr}
    """)
    
    # 4. Parameters voorbereiden (prefix om clashes te voorkomen)
    params = {}
    params.update({f"upd_{col}": val for col, val in update_values.items()})
    params.update({f"where_{col}": val for col, val in where_clauses.items()})
    
    # 5. Uitvoeren
    with engine.connect() as conn:
        result = conn.execute(query, params)
        conn.commit()
        print(f"{result.rowcount} rij(en) bijgewerkt in {table_name}.")

# Voorbeeld:
# update_table(
#     table_name="warehouses",
#     update_values={"name": "Joris de belg"},
#     where_clauses={"warehouse_id": 1}
# )


vsadmin
1 rij(en) bijgewerkt in warehouses.


In [17]:
from sqlalchemy import create_engine, text

engine = create_engine(
    f"mysql+pymysql://"
    f"{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
)
print(os.getenv('DB_USER'))


def select_from(table_name: str, where_clauses: dict = None):
    query_str = f"SELECT * FROM {table_name}"
    params = {}

    if where_clauses:
        conditions = " AND ".join(f"{col} = :{col}" for col in where_clauses)
        query_str += f" WHERE {conditions}"
        params = where_clauses

    query = text(query_str)

    with engine.connect() as conn:
        result = conn.execute(query, params)
        rows = result.fetchall()
        for row in rows:
            print(row)
        return rows

# Voorbeeld:
select_from("warehouses", {"city": "Eindhoven"})

vsadmin
(2, 'Central Warehouse', 'Racheltje 1', 'Eindhoven', '1234AB', datetime.datetime(2025, 5, 12, 11, 0, 2))


[(2, 'Central Warehouse', 'Racheltje 1', 'Eindhoven', '1234AB', datetime.datetime(2025, 5, 12, 11, 0, 2))]